In [1]:
import pandas as pd
import numpy as np
import datetime

In [2]:
from sklearn.model_selection import train_test_split
import os
import seaborn as sns
from matplotlib import pyplot as plt
import warnings
import statsmodels.api as sm
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
#from imblearn.over_sampling import SMOTE
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import LabelEncoder,MinMaxScaler

In [3]:
df_main = pd.read_csv('main.csv')
df_address = pd.read_csv('address.csv')
df_paid = pd.read_csv('paid_record.csv')

In [4]:
df_main.head()

,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,DAYS_BIRTH_CLEAN,Letter,Main_ID
0,M,Y,Y,0.0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-4542.0,1.0,1.0,0.0,0.0,NaN,2.0,09/04/1990,L,5008804
1,M,Y,Y,0.0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-4542.0,1.0,1.0,0.0,0.0,NaN,2.0,09/04/1990,G,5008805
2,M,Y,Y,0.0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-1134.0,1.0,0.0,0.0,0.0,Security staff,2.0,06/05/1964,D,5008806
3,F,N,Y,0.0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-3051.0,1.0,0.0,1.0,1.0,Sales staff,1.0,26/10/1970,I,5008808
4,F,NaN,Y,0.0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-3051.0,1.0,0.0,1.0,1.0,Sales staff,1.0,26/10/1970,Z,5008809


In [5]:
df_address.head()

,New_ID,ADDRESS
0,5008804L,"06042, CT, 31 Mitchell Road"
1,5008805G,"72703, AR, 1189 Northwest End Avenue"
2,5008806D,"72701, AR, 218 Shipley Alley"
3,5008808I,"3138 P Street Northwest, 20007, DC"
4,5008809Z,"2405 230th Street, 21122, MD"


In [6]:
df_paid.head()

,ID,MONTHS_BALANCE,STATUS
0,5001711,0,X
1,5001711,-1,0
2,5001711,-2,0
3,5001711,-3,0
4,5001712,0,C


In [7]:
df_main.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 438757 entries, 0 to 438756
Data columns (total 19 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   CODE_GENDER          427894 non-null  object 
 1   FLAG_OWN_CAR         427705 non-null  object 
 2   FLAG_OWN_REALTY      427806 non-null  object 
 3   CNT_CHILDREN         427826 non-null  float64
 4   AMT_INCOME_TOTAL     427624 non-null  float64
 5   NAME_INCOME_TYPE     427718 non-null  object 
 6   NAME_EDUCATION_TYPE  427654 non-null  object 
 7   NAME_FAMILY_STATUS   427772 non-null  object 
 8   NAME_HOUSING_TYPE    427693 non-null  object 
 9   DAYS_EMPLOYED        427784 non-null  float64
 10  FLAG_MOBIL           427690 non-null  float64
 11  FLAG_WORK_PHONE      427715 non-null  float64
 12  FLAG_PHONE           427674 non-null  float64
 13  FLAG_EMAIL           427770 non-null  float64
 14  OCCUPATION_TYPE      296952 non-null  object 
 15  CNT_FAM_MEMBERS  

In [8]:
df_main.isna().sum()

CODE_GENDER             10863
FLAG_OWN_CAR            11052
FLAG_OWN_REALTY         10951
CNT_CHILDREN            10931
AMT_INCOME_TOTAL        11133
NAME_INCOME_TYPE        11039
NAME_EDUCATION_TYPE     11103
NAME_FAMILY_STATUS      10985
NAME_HOUSING_TYPE       11064
DAYS_EMPLOYED           10973
FLAG_MOBIL              11067
FLAG_WORK_PHONE         11042
FLAG_PHONE              11083
FLAG_EMAIL              10987
OCCUPATION_TYPE        141805
CNT_FAM_MEMBERS         10934
DAYS_BIRTH_CLEAN        10715
Letter                      0
Main_ID                     0
dtype: int64

In [9]:
df_address.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 438757 entries, 0 to 438756
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   New_ID   438757 non-null  object
 1   ADDRESS  438757 non-null  object
dtypes: object(2)
memory usage: 3.3+ MB


In [10]:
df_paid.isna().sum()

ID                0
MONTHS_BALANCE    0
STATUS            0
dtype: int64

In [11]:
df_address.isna().sum()

New_ID     0
ADDRESS    0
dtype: int64

In [12]:
df_address['ADDRESS']

0                    06042, CT, 31 Mitchell Road
1           72703, AR, 1189 Northwest End Avenue
2                   72701, AR, 218 Shipley Alley
3             3138 P Street Northwest, 20007, DC
4                   2405 230th Street, 21122, MD
                           ...                  
438752            37080, TN, 7730 Wilkinson Road
438753       2815 Saint Edwards Drive, 37211, TN
438754            85306, AZ, 5434 West Beck Lane
438755                 05641, VT, 21 Hill Street
438756    73120, OK, 2316 Northwest 113th Street
Name: ADDRESS, Length: 438757, dtype: object

In [13]:
df_main.dtypes

CODE_GENDER             object
FLAG_OWN_CAR            object
FLAG_OWN_REALTY         object
CNT_CHILDREN           float64
AMT_INCOME_TOTAL       float64
NAME_INCOME_TYPE        object
NAME_EDUCATION_TYPE     object
NAME_FAMILY_STATUS      object
NAME_HOUSING_TYPE       object
DAYS_EMPLOYED          float64
FLAG_MOBIL             float64
FLAG_WORK_PHONE        float64
FLAG_PHONE             float64
FLAG_EMAIL             float64
OCCUPATION_TYPE         object
CNT_FAM_MEMBERS        float64
DAYS_BIRTH_CLEAN        object
Letter                  object
Main_ID                  int64
dtype: object

In [14]:
df_main["CODE_GENDER"].fillna('F', inplace=True)
df_main["FLAG_OWN_CAR"].fillna('N', inplace=True)
df_main["FLAG_OWN_REALTY"].fillna('Y', inplace=True)

In [15]:
#change flag Yes/No in Own Car variables to boolean
df_main.loc[df_main['FLAG_OWN_CAR'] == 'Y', 'FLAG_OWN_CAR'] = 1
df_main.loc[df_main['FLAG_OWN_CAR'] == 'N', 'FLAG_OWN_CAR'] = 0
df_main['FLAG_OWN_CAR'] = df_main['FLAG_OWN_CAR'].astype('bool')
df_main['FLAG_OWN_CAR']

0          True
1          True
2          True
3         False
4         False
          ...  
438752    False
438753     True
438754    False
438755    False
438756    False
Name: FLAG_OWN_CAR, Length: 438757, dtype: bool

In [16]:
df_main.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 438757 entries, 0 to 438756
Data columns (total 19 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   CODE_GENDER          438757 non-null  object 
 1   FLAG_OWN_CAR         438757 non-null  bool   
 2   FLAG_OWN_REALTY      438757 non-null  object 
 3   CNT_CHILDREN         427826 non-null  float64
 4   AMT_INCOME_TOTAL     427624 non-null  float64
 5   NAME_INCOME_TYPE     427718 non-null  object 
 6   NAME_EDUCATION_TYPE  427654 non-null  object 
 7   NAME_FAMILY_STATUS   427772 non-null  object 
 8   NAME_HOUSING_TYPE    427693 non-null  object 
 9   DAYS_EMPLOYED        427784 non-null  float64
 10  FLAG_MOBIL           427690 non-null  float64
 11  FLAG_WORK_PHONE      427715 non-null  float64
 12  FLAG_PHONE           427674 non-null  float64
 13  FLAG_EMAIL           427770 non-null  float64
 14  OCCUPATION_TYPE      296952 non-null  object 
 15  CNT_FAM_MEMBERS  

In [17]:
#change flag Yes/No in OWN_REALTY  variables to boolean
df_main.loc[df_main['FLAG_OWN_REALTY'] == 'Y', 'FLAG_OWN_REALTY'] = 1
df_main.loc[df_main['FLAG_OWN_REALTY'] == 'N', 'FLAG_OWN_REALTY'] = 0
df_main['FLAG_OWN_REALTY'] = df_main['FLAG_OWN_REALTY'].astype('bool')
df_main['FLAG_OWN_REALTY']

0          True
1          True
2          True
3          True
4          True
          ...  
438752    False
438753     True
438754     True
438755     True
438756     True
Name: FLAG_OWN_REALTY, Length: 438757, dtype: bool

In [18]:
df_main.FLAG_OWN_REALTY.unique()

array([ True, False])

In [19]:

del(df_main["FLAG_MOBIL"])

def obtener_lista_variables(dataset):
    lista_numericas=[]
    lista_boolean=[]
    lista_categoricas=[]

    for i in dataset:
        if    (dataset[i].dtype.kind in ("f","i")):
              lista_numericas.append(i)
        elif  (dataset[i].dtype.kind in ("f","i", "O")) and len(dataset[i].unique())== 2:
              lista_boolean.append(i)
        elif  (dataset[i].dtype.kind == "O"):
              lista_categoricas.append(i)
                
    return lista_numericas, lista_boolean, lista_categoricas
lista_numericas, lista_boolean, lista_categoricas= obtener_lista_variables(df_main)

for i in lista_numericas:
    median_columna = df_main[i].median()
    df_main[i].fillna(median_columna, inplace=True)
    
df_main["NAME_INCOME_TYPE"].fillna('Working', inplace=True)
df_main["NAME_EDUCATION_TYPE"].fillna('Secondary / secondary special', inplace=True)
df_main["NAME_FAMILY_STATUS"].fillna('Married', inplace=True)
df_main["NAME_HOUSING_TYPE"].fillna('House / apartment', inplace=True)
df_main["OCCUPATION_TYPE"].fillna('Laborers', inplace=True)

In [20]:
#cast DAYS_BIRTH_CLEAN as datetime
df_main['BIRTH_YEAR'] = pd.to_datetime(df_main['DAYS_BIRTH_CLEAN'], format='%d/%m/%Y').dt.year

In [21]:
df_main['AGE'] = 2023 - df_main['BIRTH_YEAR']

In [22]:
df_main['AGE']

0         33.0
1         33.0
2         59.0
3         53.0
4         53.0
          ... 
438752    32.0
438753    45.0
438754    49.0
438755    58.0
438756    34.0
Name: AGE, Length: 438757, dtype: float64

In [23]:
df_main.loc[df_main['DAYS_EMPLOYED'] < 0, 'DAYS_EMPLOYED']

0        -4542.0
1        -4542.0
2        -1134.0
3        -3051.0
4        -3051.0
           ...  
438752   -3331.0
438753   -5941.0
438754    -880.0
438755    -885.0
438756    -783.0
Name: DAYS_EMPLOYED, Length: 365386, dtype: float64

In [24]:
df_main.loc[df_main['DAYS_EMPLOYED'] >= 0, 'DAYS_EMPLOYED'] = 0

In [25]:
df_main.loc[df_main['DAYS_EMPLOYED'] < 0, 'DAYS_EMPLOYED'] = 1

Merge

In [26]:
df_paid = df_paid[['ID','STATUS']]

In [27]:
df_main = df_main[['Main_ID','CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'AGE', 'DAYS_EMPLOYED','CNT_FAM_MEMBERS']]

In [28]:
df_final = df_main.merge(df_paid, left_on = 'Main_ID', right_on = 'ID')

In [29]:
df_final.head()

,Main_ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AGE,DAYS_EMPLOYED,CNT_FAM_MEMBERS,ID,STATUS
0,5008804,M,True,True,0.0,427500.0,33.0,1.0,2.0,5008804,C
1,5008804,M,True,True,0.0,427500.0,33.0,1.0,2.0,5008804,C
2,5008804,M,True,True,0.0,427500.0,33.0,1.0,2.0,5008804,C
3,5008804,M,True,True,0.0,427500.0,33.0,1.0,2.0,5008804,C
4,5008804,M,True,True,0.0,427500.0,33.0,1.0,2.0,5008804,C


In [30]:
df_final['STATUS'].value_counts()

C    330916
0    291813
X    146421
1      8797
5      1527
2       801
3       286
4       214
Name: STATUS, dtype: int64

In [31]:
df_final['TARGET'] = df_final['STATUS'].isin(['C', 'X', '0'])

In [32]:
df_final['TARGET'].value_counts()

True     769150
False     11625
Name: TARGET, dtype: int64

In [33]:
df_final['TARGET'] = df_final['TARGET'].astype('int32')

In [34]:
df_final['TARGET'].unique()

array([1, 0])

In [35]:
corr = df_final.corr()
corr.style.background_gradient()

,Main_ID,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AGE,DAYS_EMPLOYED,CNT_FAM_MEMBERS,ID,TARGET
Main_ID,1.000000,-0.011518,-0.096377,0.025114,-0.026806,-0.040319,0.023196,0.028384,1.000000,-0.008423
FLAG_OWN_CAR,-0.011518,1.000000,-0.020953,0.103701,0.209770,-0.156043,0.153027,0.151223,-0.011518,0.002740
FLAG_OWN_REALTY,-0.096377,-0.020953,1.000000,-0.009610,0.035497,0.130454,-0.088906,-0.015838,-0.096377,0.008815
CNT_CHILDREN,0.025114,0.103701,-0.009610,1.000000,0.039777,-0.350262,0.223481,0.868454,0.025114,-0.005315
AMT_INCOME_TOTAL,-0.026806,0.209770,0.035497,0.039777,1.000000,-0.078530,0.173683,0.031101,-0.026806,-0.006386
AGE,-0.040319,-0.156043,0.130454,-0.350262,-0.078530,1.000000,-0.602693,-0.321654,-0.040319,0.014540
DAYS_EMPLOYED,0.023196,0.153027,-0.088906,0.223481,0.173683,-0.602693,1.000000,0.220868,0.023196,-0.007351
CNT_FAM_MEMBERS,0.028384,0.151223,-0.015838,0.868454,0.031101,-0.321654,0.220868,1.000000,0.028384,-0.001784
ID,1.000000,-0.011518,-0.096377,0.025114,-0.026806,-0.040319,0.023196,0.028384,1.000000,-0.008423
TARGET,-0.008423,0.002740,0.008815,-0.005315,-0.006386,0.014540,-0.007351,-0.001784,-0.008423,1.000000


Get dummies

In [36]:
def obtener_lista_variables(dataset):
    lista_numericas=[]
    lista_boolean=[]
    lista_categoricas=[]

    for i in dataset:
        if    (dataset[i].dtype.kind in ("f","i")) and len(dataset[i].unique())!= 2:
              lista_numericas.append(i)
        elif  (dataset[i].dtype.kind in ("f","i", "O")) and len(dataset[i].unique())== 2:
              lista_boolean.append(i)
        elif  (dataset[i].dtype.kind == "O"):
              lista_categoricas.append(i)
                
    return lista_numericas, lista_boolean, lista_categoricas
lista_numericas, lista_boolean, lista_categoricas= obtener_lista_variables(df_final)

In [37]:
df_final= pd.get_dummies(data=df_final, columns=lista_categoricas)
df_final= pd.get_dummies(data=df_final, columns=lista_boolean)

In [38]:
minMaxResultado = MinMaxScaler()
for i in lista_numericas:
    df_final[i] = minMaxResultado.fit_transform(df_final[i].values.reshape(-1,1))

In [39]:
median_columna = df_final['AGE'].median()
df_final['AGE'].fillna(median_columna, inplace=True)

In [40]:
df_final.head()

,Main_ID,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AGE,CNT_FAM_MEMBERS,ID,STATUS_0,STATUS_1,...,STATUS_4,STATUS_5,STATUS_C,STATUS_X,CODE_GENDER_F,CODE_GENDER_M,DAYS_EMPLOYED_0.0,DAYS_EMPLOYED_1.0,TARGET_0,TARGET_1
0,0.0,True,True,0.0,0.258721,0.25,0.052632,0.0,0,0,...,0,0,1,0,0,1,0,1,0,1
1,0.0,True,True,0.0,0.258721,0.25,0.052632,0.0,0,0,...,0,0,1,0,0,1,0,1,0,1
2,0.0,True,True,0.0,0.258721,0.25,0.052632,0.0,0,0,...,0,0,1,0,0,1,0,1,0,1
3,0.0,True,True,0.0,0.258721,0.25,0.052632,0.0,0,0,...,0,0,1,0,0,1,0,1,0,1
4,0.0,True,True,0.0,0.258721,0.25,0.052632,0.0,0,0,...,0,0,1,0,0,1,0,1,0,1


In [41]:
df_final.isnull().sum()

Main_ID              0
FLAG_OWN_CAR         0
FLAG_OWN_REALTY      0
CNT_CHILDREN         0
AMT_INCOME_TOTAL     0
AGE                  0
CNT_FAM_MEMBERS      0
ID                   0
STATUS_0             0
STATUS_1             0
STATUS_2             0
STATUS_3             0
STATUS_4             0
STATUS_5             0
STATUS_C             0
STATUS_X             0
CODE_GENDER_F        0
CODE_GENDER_M        0
DAYS_EMPLOYED_0.0    0
DAYS_EMPLOYED_1.0    0
TARGET_0             0
TARGET_1             0
dtype: int64

In [42]:
#export csv
df_final.to_csv('final_df.csv', sep=';')

Visualización

In [43]:
import sweetviz

In [44]:
report = sweetviz.analyze(df_final, "TARGET")

KeyError: "Feature 'TARGET' was specified as TARGET, but is NOT FOUND in the dataframe (watch case-sensitivity?)."

In [ ]:
report.show_html()

Modelling

In [ ]:
target = 'TARGET'
features = df_final.columns[df_final.columns != target]

y = df_final[target]
X = df_final[features]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.30, random_state=42)

In [ ]:
sm = SMOTE(random_state = 2)
X_train, y_train = sm.fit_resample(X_train, y_train)
os_data_X,os_data_y= sm.fit_resample(X_train, y_train)
y_train.value_counts()

In [ ]:
LR = LogisticRegression()
LR.fit(X_train, y_train)

In [ ]:
y_test = LR.predict(X_test)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(LR.score(X_test, y_test)))

In [ ]:
logit_roc_auc = roc_auc_score(y_test, LR.predict(X_test))
fpr, tpr, thresholds = roc_curve(y_test, LR.predict_proba(X_test)[:,1])
plt.figure()
plt.plot(fpr, tpr, label='Logistic Regression (area = %0.2f)' % logit_roc_auc)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.savefig('Log_ROC')
plt.show()

In [ ]:
#GaussianNB
gnb = GaussianNB()
gnb.fit(X_train, y_train)
y_pred = gnb.predict(X_test)

In [ ]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))